<a href="https://colab.research.google.com/github/soutrik71/pytorch_classics/blob/main/APTorch7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The focus of this notebook is to implement LSTM for sequence generation tasks
1. Text Generation using LSTM Networks (Character-based RNN)
2. Text Generation using PyTorch LSTM Networks (Character Embeddings)
3. Sequence generation at word level using LSTM Networks (Word Embeddings)
3. Application of pre-trained embedding models for the better representation of words

In [78]:
# !pip install portalocker
# !pip install torchview
# !pip install torcheval
# !pip install scikit-plot
# !pip install lime

In [79]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [80]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchtext
from torch.utils.data import DataLoader, TensorDataset
from torchtext import data
from torchtext import datasets
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import re
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset
from torchsummary import summary
from torchview import draw_graph
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from torcheval.metrics import MulticlassAccuracy,BinaryAccuracy
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
import scikitplot as skplt

In [81]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [82]:
# Set manual seed since nn.Parameter are randomly initialzied
set_seed(42)
# Set device cuda for GPU if it's available otherwise run on the CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 1024
epochs = 10
lr = 1e-4
embedding = False

Random seed set as 42
cuda


## Basic DataPrep

In this section, we undertake data preparation for training the neural network using a character-based approach. Specifically, we adopt a fixed sequence length of 100 characters, where the network's task is to predict the subsequent character given this sequence. Employing an embeddings technique, we encode text data by assigning a unique real-valued vector to each character.

The data preparation procedure is as follows:

* Loading Text Examples and Creating Vocabulary: Iterate through all text examples to construct a vocabulary, mapping each character to a distinct integer index. This vocabulary
facilitates character representation in a numerical format.

* Organizing Data with a Sliding Window: Implement a sliding window mechanism to organize the data. For every text example, we slide a window of 100 characters. The first 100 characters serve as input features (X), while the 101st character becomes the target value (Y). This process continues by shifting the window one character at a time until the end of the text example.

* Conversion to Integer Indices: Retrieve integer indices corresponding to characters in both data features and target values based on the previously constructed vocabulary. This step transforms characters into their corresponding numerical representations.

* Embeddings Assignment: Each unique integer index, representing a specific character in the data features, is associated with a real-valued vector known as an embedding. These embeddings provide a continuous representation of characters, facilitating numerical computation within the neural network. This is optional

### Data loading

In [83]:
train_dataset, valid_dataset, test_dataset = datasets.PennTreebank()

In [84]:
next(iter(train_dataset.shuffle()))

'instead new york city police seized the stolen goods and mr. <unk> avoided jail'

In [85]:
def info(x):
  return len(x)

elem_ls = list(train_dataset.map(info))

/home/azureuser/.local/lib/python3.10/site-packages/torch/utils/data/datapipes/iter/combining.py:337: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [86]:
print(len(elem_ls)) # total 42k elements
print(max(elem_ls)) # max length of each element
print(min(elem_ls)) # min length of each element

42068
518
2


We construct a vocabulary of unique characters using build_vocab_from_iterator() from torchtext's 'vocab' sub-module. Our custom function build_vocabulary() serves as an iterator, looping through datasets and examples to yield character lists. Special handling ensures the '<unk>' token, representing unknown characters, is counted as a single token rather than individual characters.






In [87]:
def build_vocabulary(datasets):
  for dataset in datasets:
    for text in dataset:

      if "unk" in text:
        texts = text.split("<unk>")
        total = list(texts[0].lower())
        for t in texts[1:]:
            total.extend(["<unk>", ] + list(t.lower()))
        yield total

      else:
        yield list(text.lower())

In [88]:
vocab = build_vocab_from_iterator(build_vocabulary([train_dataset, valid_dataset, test_dataset]), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [89]:
len(vocab)

47

In [90]:
print(vocab.get_itos()) # character level tokenization

['<unk>', ' ', 'e', 't', 'a', 'n', 'o', 'i', 's', 'r', 'h', 'l', 'd', 'c', 'u', 'm', 'f', 'p', 'g', 'y', 'b', 'w', 'v', 'k', '.', "'", 'x', 'j', '$', '-', 'q', 'z', '&', '0', '1', '9', '3', '#', '2', '8', '5', '\\', '7', '6', '/', '4', '*']


In [91]:
print(vocab.get_stoi()) # dictionary mapping token to indices

{'4': 45, '/': 44, '7': 42, '8': 39, '2': 38, '#': 37, '9': 35, '1': 34, 'z': 31, 'q': 30, '-': 29, '6': 43, '3': 36, 'r': 9, 's': 8, 'd': 12, 'k': 23, 'n': 5, 'h': 10, '*': 46, 'u': 14, '0': 33, 'p': 17, 't': 3, 'i': 7, '\\': 41, '5': 40, 'a': 4, 'e': 2, 'j': 27, '&': 32, 'v': 22, 'o': 6, '<unk>': 0, '.': 24, 'c': 13, 'm': 15, 'f': 16, 'l': 11, 'g': 18, 'y': 19, 'b': 20, 'w': 21, ' ': 1, 'x': 26, "'": 25, '$': 28}


Preparing the sequential data for training with sliding window approach and window size of 10 characters

In [92]:
seq_len = 35
train_records_max = 5000
X_train_full, y_train_full = [], []
X_val_full , y_val_full = [], []

In [93]:
# train data prep
for idex, text in enumerate(train_dataset):
  print(text)
  print("\n")
  for i in range(len(text) - seq_len):
    inp_rec = list(text[i:i+seq_len].lower())
    op_rec = text[i+seq_len].lower()

    if len(op_rec) == 0:
      break

    X_train_full.append(vocab(inp_rec))
    y_train_full.append(vocab[op_rec])

  if idex > train_records_max:
    break

aer banknote berlitz calloway centrust cluett fromstein gitano guterman hydro-quebec ipo kia memotec mlx nahb punts rake regatta rubens sim snack-food ssangyong swapo wachter


pierre <unk> N years old will join the board as a nonexecutive director nov. N


mr. <unk> is chairman of <unk> n.v. the dutch publishing group


rudolph <unk> N years old and former chairman of consolidated gold fields plc was named a nonexecutive director of this british industrial conglomerate


a form of asbestos once used to make kent cigarette filters has caused a high percentage of cancer deaths among a group of workers exposed to it more than N years ago researchers reported


the asbestos fiber <unk> is unusually <unk> once it enters the <unk> with even brief exposures to it causing symptoms that show up decades later researchers said


<unk> inc. the unit of new york-based <unk> corp. that makes kent cigarettes stopped using <unk> in its <unk> cigarette filters in N


although preliminary findings were


the prevailing interpretation of the clause on capitol hill is that it gives congress an <unk> veto over every <unk> action of the president through the ability to <unk> funding


this interpretation was officially endorsed by congress in N in the iran-contra report


as <unk> of congressional power understand a power of the <unk> so broadly <unk> would <unk> the presidency and <unk> the principle of separation of powers


it is not supported by the text or history of the constitution


the <unk> hardly discussed the appropriations clause at the constitutional convention of N according to madison 's notes


to the extent they did their concern was to ensure fiscal accountability


moreover the <unk> believed that the nation needed a <unk> executive with the independence and resources to perform the executive functions that the confederation congress had performed poorly under the articles of confederation


it would <unk> that objective if the appropriations clause technically a <unk>

In [94]:
print(len(X_train_full))
print(len(y_train_full))

423585
423585


In [95]:
# validation dataset prep
for idex, text in enumerate(valid_dataset):
  print(text)
  print("\n")
  for i in range(len(text) - seq_len):
    inp_rec = list(text[i:i+seq_len].lower())
    op_rec = text[i+seq_len].lower()

    if len(op_rec) == 0:
      break

    X_val_full.append(vocab(inp_rec))
    y_val_full.append(vocab[op_rec])

consumers may want to move their telephones a little closer to the tv set


<unk> <unk> watching abc 's monday night football can now vote during <unk> for the greatest play in N years from among four or five <unk> <unk>


two weeks ago viewers of several nbc <unk> consumer segments started calling a N number for advice on various <unk> issues


and the new syndicated reality show hard copy records viewers ' opinions for possible airing on the next day 's show


interactive telephone technology has taken a new leap in <unk> and television programmers are racing to exploit the possibilities


eventually viewers may grow <unk> with the technology and <unk> the cost


but right now programmers are figuring that viewers who are busy dialing up a range of services may put down their <unk> control <unk> and stay <unk>


we 've been spending a lot of time in los angeles talking to tv production people says mike parks president of call interactive which supplied technology for both abc sports 



art indexes track winners not losers


but art that has fallen sharply in value is rarely put up for sale


also at any of sotheby 's auctions of old masters roughly one-third to <unk> of what is offered does n't sell at any price


it 's not that there are n't any bids but the bids do n't meet the minimum reserve prices set by the sellers


in january the <unk> painting that now hangs at centrust was expected to bring no more than $ N at auction until mr. paul came along with his $ N million


mr. hall of the <unk> gallery says $ N million would have been an impossible price for anyone to ask for a <unk> four years ago


but from his <unk> point it is n't that mr. paul a customer of his too <unk> for the work a <unk> painting by an artist who is not a household word


the painting is N feet wide seven feet high


rather it just shows things have changed


mr. paul boasts that he spotted bargains in old masters just before they took an upward turn


they went up N N last year and the

In [96]:
print(len(X_val_full))
print(len(y_val_full))

273982
273982


In [97]:
X_train = torch.tensor(X_train_full, dtype=torch.float32)
y_train = torch.tensor(y_train_full)
print(f"The shape of X_train is {X_train.shape}") # n records with k elements in each
print(f"The shape of Y_train is {y_train.shape}") # n records with 1 element in each

The shape of X_train is torch.Size([423585, 35])
The shape of Y_train is torch.Size([423585])


In [98]:
X_val = torch.tensor(X_val_full, dtype=torch.float32)
y_val = torch.tensor(y_val_full)
print(f"The shape of X_train is {X_val.shape}") # n records with k elements in each
print(f"The shape of Y_train is {y_val.shape}") # n records with 1 element in each

The shape of X_train is torch.Size([273982, 35])
The shape of Y_train is torch.Size([273982])


In [99]:
if not embedding:
  X_train = X_train.unsqueeze(dim=-1)
  X_val = X_val.unsqueeze(dim=-1)

Dataloader part

In [100]:
vectorized_train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(vectorized_train_dataset, batch_size=1024, shuffle=False)

vectorized_valid_dataset = TensorDataset(X_val, y_val)
valid_loader = DataLoader(vectorized_valid_dataset, batch_size=1024, shuffle=False)

In [101]:
for x, y in train_loader:
  print(x.shape)
  print(y.shape)
  break

torch.Size([1024, 35, 1])
torch.Size([1024])


## Modelling Building using Character based RNN

The network includes 2 LSTM layers with an output size of 256 each, followed by a linear layer. Stacking these LSTM layers enhances sequence learning. The output of the second LSTM layer feeds into the linear layer, whose output units match the vocabulary size

In [102]:
hidden_dim = 256
n_layers=2

class LSTMTextGenerator(nn.Module):
    def __init__(self):
        super(LSTMTextGenerator, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(vocab))

    def forward(self, X_batch):
      # init weights
      hidden = torch.randn(n_layers, len(X_batch), hidden_dim).to(device)
      carry = torch.randn(n_layers, len(X_batch), hidden_dim).to(device)

      output, (hidden, carry) = self.lstm(X_batch, (hidden, carry))
      return self.linear(output[:,-1])

In [103]:
text_generator_lstm = LSTMTextGenerator().to(device)

In [104]:
for layer in text_generator_lstm.children():
    print("Layer : {}".format(layer))
    print("Parameters : ")
    for param in layer.parameters():
        print(param.shape)
    print("\n")

Layer : LSTM(1, 256, num_layers=2, batch_first=True)
Parameters : 
torch.Size([1024, 1])
torch.Size([1024, 256])
torch.Size([1024])
torch.Size([1024])
torch.Size([1024, 256])
torch.Size([1024, 256])
torch.Size([1024])
torch.Size([1024])


Layer : Linear(in_features=256, out_features=47, bias=True)
Parameters : 
torch.Size([47, 256])
torch.Size([47])




In [105]:
def train_module(model:torch.nn.Module,
                 device:torch.device,
                 train_dataloader:torch.utils.data.DataLoader ,
                 optimizer:torch.optim.Optimizer,
                 criterion:torch.nn.Module,
                 metric,
                 train_losses:list,
                 train_metrics:list):

  # setting model to train mode
  model.train()
  pbar = tqdm(train_dataloader)

  # batch metrics
  train_loss = 0
  processed_batch = 0

  for idx, (data,label) in enumerate(pbar):
    # setting up device
    data = data.to(device)
    label = label.to(device)

    # forward pass output
    preds = model(data)

    # calc loss
    loss = criterion(preds, label)
    train_loss += loss.item()
    # print(f"training loss for batch {idx} is {loss}")

    # backpropagation
    optimizer.zero_grad() # flush out  existing grads
    loss.backward() # back prop of weights wrt loss
    optimizer.step() # optimizer step -> minima

    #updating batch count
    processed_batch += 1

    pbar.set_description(f"Avg Train Loss: {train_loss/processed_batch}")

  # updating epoch metrics
  train_losses.append(train_loss/processed_batch)

  return train_losses


In [106]:
def test_module(model:torch.nn.Module,
                device:torch.device,
                test_dataloader:torch.utils.data.DataLoader,
                criterion:torch.nn.Module,
                metric,
                test_losses,
                test_metrics):
  # setting model to eval mode
  model.eval()
  pbar = tqdm(test_dataloader)

  # batch metrics
  test_loss = 0
  processed_batch = 0

  with torch.inference_mode():
    for idx, (data,label) in enumerate(pbar):
      data , label = data.to(device), label.to(device)
      # predictions
      preds = model(data)
      # print(preds.shape)
      # print(label.shape)

      #loss calc
      loss = criterion(preds, label)
      test_loss += loss.item()

      #updating batch count
      processed_batch += 1

      pbar.set_description(f"Avg Test Loss: {test_loss/processed_batch}")

    # updating epoch metrics
    test_losses.append(test_loss/processed_batch)

  return test_losses

In [107]:
optimizer = optim.Adam(text_generator_lstm.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [108]:
%%time

# Place holders----
train_losses = []
test_losses = []

for epoch in range(0,epochs):
  print(f'Epoch {epoch}')
  train_losses = train_module(text_generator_lstm, device, train_loader, optimizer, criterion, None, train_losses, None)
  test_losses = test_module(text_generator_lstm, device, valid_loader, criterion, None, test_losses, None)

Epoch 0


Avg Train Loss: 3.6084525817778053:   9%|███████████████████████████████████████████████████████████████████████████████████████▏                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 39/414 [00:01<00:17, 22.03it/s]

Avg Test Loss: 2.8952118158340454: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 38.14it/s]


Epoch 1


Avg Train Loss: 2.8997944259643553:   6%|███████████████████████████████████████████████████████▊                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     | 25/414 [00:01<00:17, 21.80it/s]


KeyboardInterrupt: 

## Evaluation
The logic starts with the initial randomly selected sequence and makes the next character prediction. It then removes the first character from the sequence and adds a newly predicted character at the end. Then, it makes another prediction and the process repeats for 100 characters.

In [ ]:
idx = random.randint(0, len(X_train))
pattern = X_train[idx].numpy().astype(int).flatten().tolist() # list of tokens matched with torch text
print("Initial Pattern : {}".format("".join(vocab.lookup_tokens(pattern))))

Initial Pattern : iami fla. company that packages and


In [ ]:
generated_text = []
# genererate 100 characters and also offsetting one character after every iterations to maintain the seq length
for i in range(100):
  batch = torch.tensor(pattern, dtype=torch.float32).reshape(1, seq_len, 1).to(device)
  model_op = text_generator_lstm(batch)
  # print(model_op.shape) # 47 is the vocab size
  predicted_index = model_op.argmax(dim=-1).squeeze().cpu().item()
  generated_text.append(predicted_index) ## Add token index to result
  pattern.append(predicted_index) ## Add token index to original pattern
  pattern = pattern[1:] ## Resize pattern to bring again to seq_length l


In [ ]:
print("Generated Text : {}".format("".join(vocab.lookup_tokens(generated_text))))

Generated Text :  the the the the the the the the the the the the the the the the the the the the the the the the the


The model is producing some random text but english text but in repeations
and even after mutiple training yet it produces similar random text

## Model Building using self trained embeddings

We have used character-based approach for our case which means that our network takes a list of characters as input and returns the next character that it thinks should come next. We can also design models that take a list of words as input and predicts the next word. For encoding text data, we have used character embeddings approach which assigns a real-valued vector to each token (character)

Network Architecture:

* Embedding Layer: 100 embedding length, input (batch_size, seq_length), output (batch_size, seq_length, 100).
* LSTM Layer 1 & 2: 256 hidden dimensions, input (batch_size, seq_length, embed_len), output (batch_size, seq_length, 256).
* Linear Layer: Output units match vocabulary length, input (batch_size, seq_length, 256), output (batch_size, vocab_len).
* Embedding Layer:

  Utilizes Embedding() constructor with vocab length and 100 embedding length.\
  Transforms input shape to (batch_size, seq_length, embed_len).

* LSTM Layers:

  LSTM Layer 1 processes embedding output with 256 hidden dimensions.\
  LSTM Layer 2 processes LSTM 1 output with 256 hidden dimensions.

* Linear Layer:\
  Transforms LSTM 2 output to (batch_size, vocab_len), representing predictions.

* Initialization & Verification:\
  Initialized network and examined weights/biases.\
  Conducted forward pass with sample data for validation.

In [ ]:
X_train = torch.tensor(X_train_full, dtype=torch.int64)
y_train = torch.tensor(y_train_full)
print(f"The shape of X_train is {X_train.shape}") # n records with k elements in each
print(f"The shape of Y_train is {y_train.shape}") # n records with 1 element in each

The shape of X_train is torch.Size([423585, 35])
The shape of Y_train is torch.Size([423585])


In [ ]:
X_val = torch.tensor(X_val_full, dtype=torch.int64)
y_val = torch.tensor(y_val_full)
print(f"The shape of X_train is {X_val.shape}") # n records with k elements in each
print(f"The shape of Y_train is {y_val.shape}") # n records with 1 element in each

The shape of X_train is torch.Size([273982, 35])
The shape of Y_train is torch.Size([273982])


In [ ]:
# new data loader
vectorized_train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(vectorized_train_dataset, batch_size=1024, shuffle=False)

vectorized_valid_dataset = TensorDataset(X_val, y_val)
valid_loader = DataLoader(vectorized_valid_dataset, batch_size=1024, shuffle=False)

In [ ]:
embed_len = 100
hidden_dim = 256
n_layers=2

class LSTMTextGenerator_Embed(nn.Module):
    def __init__(self):
        super(LSTMTextGenerator_Embed, self).__init__()
        self.word_embedding = nn.Embedding(num_embeddings= 47, embedding_dim=embed_len)
        self.lstm = nn.LSTM(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(vocab))

    def forward(self, X_batch):
        embeddings = self.word_embedding(X_batch)

        hidden, carry = torch.randn(n_layers, len(X_batch), hidden_dim).to(device), torch.randn(n_layers, len(X_batch), hidden_dim).to(device)
        output, (hidden, carry) = self.lstm(embeddings, (hidden, carry))
        return self.linear(output[:,-1])

In [ ]:
text_generator_lstm_embd = LSTMTextGenerator_Embed().to(device)

In [ ]:
optimizer = optim.Adam(text_generator_lstm_embd.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
%%time

# Place holders----
train_losses = []
test_losses = []

for epoch in range(0,epochs):
  print(f'Epoch {epoch}')
  train_losses = train_module(text_generator_lstm_embd, device, train_loader, optimizer, criterion, None, train_losses, None)
  test_losses = test_module(text_generator_lstm_embd, device, valid_loader, criterion, None, test_losses, None)

Epoch 0


  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          | 0/414 [00:00<?, ?it/s]

Avg Test Loss: 2.4405801331818995: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 37.05it/s]


Epoch 1


Avg Test Loss: 2.157608670974845: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 37.33it/s]


...

Epoch 8


Avg Test Loss: 1.6765732146910768: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 37.55it/s]


Epoch 9


Avg Test Loss: 1.6440599760902461: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268/268 [00:07<00:00, 37.76it/s]

CPU times: user 4min 34s, sys: 2.49 s, total: 4min 37s
Wall time: 4min 35s


In [ ]:
idx = random.randint(0, len(X_train))
pattern = X_train[idx].numpy().astype(int).flatten().tolist() # list of tokens matched with torch text
print("Initial Pattern : {}".format("".join(vocab.lookup_tokens(pattern))))

Initial Pattern : of the plant 's construction cost w


In [ ]:
generated_text = []
# genererate 100 characters and also offsetting one character after every iterations to maintain the seq length
for i in range(100):
  batch = torch.tensor(pattern, dtype=torch.int64).reshape(1, seq_len).to(device)
  model_op = text_generator_lstm_embd(batch)
  # print(model_op.shape) # 47 is the vocab size
  predicted_index = model_op.argmax(dim=-1).squeeze().cpu().item()
  generated_text.append(predicted_index) ## Add token index to result
  pattern.append(predicted_index) ## Add token index to original pattern
  pattern = pattern[1:] ## Resize pattern to bring again to seq_length l


In [ ]:
print("Generated Text : {}".format("".join(vocab.lookup_tokens(generated_text))))

Generated Text : ith the <unk>unk<unk> and the <unk>unk<unk> and the <unk>unk<unk> and the <unk>unk<unk> and the <unk>unk<unk> and the <unk>unk<unk> and the <unk>unk<unk> an


References:

https://coderzcolumn.com/tutorials/artificial-intelligence/text-generation-using-pytorch-lstm-networks-and-character-embeddings\
https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-text-generation-using-lstm-networks

## Text Generation Word level

Shortcomings of Character-Level Text Generation:

Lack of Context: Operates at a granular level, focusing on individual characters, which hinders the ability to maintain broader context necessary for coherent text generation.
Prone to Random Output: In longer sequences, more susceptible to generating gibberish or nonsensical text due to the absence of word-level semantics.

Promise of Word Level Text Generation:

Improved Contextual Understanding: Operates at a higher linguistic level, capturing relationships between words, phrases, and sentences for more coherent text generation, suitable for tasks requiring longer sequences.
Enhanced Readability: Produces text that is more readable and human-like by adhering to linguistic conventions, making it suitable for applications like story generation and content creation.

In [ ]:
with open('data/alice/alice.txt', 'r', encoding='utf-8') as file:
  train_text = file.read()


The process would primarily involve:
- First, we will prepare the dataset. This includes the preparation of vocabulary, a dictionary for mapping words to integers, and a reverse dictionary as well.
- Second, we need to prepare the data loaders where the input will be sentences to a certain length and the targets will be the same sentences but shifted one place to the right.
- Third, we need to prepare the LSTM model.
- Finally, we will carry out the training and inference using the trained model.

In [ ]:
len(train_text)

145190

#### Preprocessing of text io rudimentary way

In [ ]:
from collections import Counter

In [ ]:
words = train_text.split()
words_counter = Counter(words)

In [ ]:
print(len(words))
print(words_counter.most_common(10))

26563
[('the', 1510), ('and', 715), ('to', 703), ('a', 606), ('of', 493), ('she', 484), ('said', 416), ('it', 348), ('in', 347), ('was', 328)]


In [ ]:
# unique word count
vocab_ls = words_counter.keys()
vocab_size = len(vocab_ls)

In [ ]:
def words_to_index():
    op = {word: i + 1 for i, word in enumerate(vocab_ls)}
    op["<unk>"] = 0

    return op


def index_to_word():
    op = words_to_index()
    return {i: word for word, i in op.items()}

In [ ]:
words_to_idx = words_to_index()
idx_to_word = index_to_word()

In [ ]:
words_to_idx['<unk>']

0

In [ ]:
idx_to_word[0]

'<unk>'

In [126]:
# we are creating each sample of 64 words with first 63 is training and last one is always testing
# next set of words starts based on the step size
SEQUENCE_LENGTH = 64
samples = [words[i:i+SEQUENCE_LENGTH+1] for i in range(0,len(words)-SEQUENCE_LENGTH,1)]

In [127]:
print(len(samples))

26499


In [128]:
samples[0]

['\ufeffProject',
 "Gutenberg's",
 "Alice's",
 'Adventures',
 'in',
 'Wonderland,',
 'by',
 'Lewis',
 'Carroll',
 'This',
 'eBook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever.',
 'You',
 'may',
 'copy',
 'it,',
 'give',
 'it',
 'away',
 'or',
 're-use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'Project',
 'Gutenberg',
 'License',
 'included',
 'with',
 'this',
 'eBook',
 'or',
 'online',
 'at',
 'www.gutenberg.org',
 'Title:',
 "Alice's",
 'Adventures',
 'in',
 'Wonderland',
 'Author:',
 'Lewis',
 'Carroll',
 'Posting',
 'Date:',
 'June',
 '25,']

In [129]:
samples[1]

["Gutenberg's",
 "Alice's",
 'Adventures',
 'in',
 'Wonderland,',
 'by',
 'Lewis',
 'Carroll',
 'This',
 'eBook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyone',
 'anywhere',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrictions',
 'whatsoever.',
 'You',
 'may',
 'copy',
 'it,',
 'give',
 'it',
 'away',
 'or',
 're-use',
 'it',
 'under',
 'the',
 'terms',
 'of',
 'the',
 'Project',
 'Gutenberg',
 'License',
 'included',
 'with',
 'this',
 'eBook',
 'or',
 'online',
 'at',
 'www.gutenberg.org',
 'Title:',
 "Alice's",
 'Adventures',
 'in',
 'Wonderland',
 'Author:',
 'Lewis',
 'Carroll',
 'Posting',
 'Date:',
 'June',
 '25,',
 '2008']